In [ ]:
# Set Directories

MAIN_DIR = "/gws/nopw/j04/ai4er/users/trr26/earthquake-predictability"
DATA_DIR = "/gws/nopw/j04/ai4er/users/trr26/earthquake-predictability/data/gtc_quakes_data"
p4581_FILE_PATH = f"{DATA_DIR}/labquakes/Marone/p4581/p4581.txt"

# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from math import sqrt
from scipy.io import loadmat
import scipy
import ipywidgets as widgets
from IPython.display import display

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader 
from tqdm.auto import tqdm
import random
import os

import sys
sys.path.append(MAIN_DIR)
import utils

In [ ]:
# Check CUDA Availability

if torch.cuda.is_available():  
    dev = "cuda" 
    map_location=None
    print(f"Total number of GPUs available: {torch.cuda.device_count()}")
    !nvidia-smi
else:  
    dev = "cpu"  
    map_location='cpu'
    print("No GPU available.")
device = torch.device(dev)

In [ ]:
with open(p4581_FILE_PATH, "r") as file:
    df = pd.read_csv(file, delim_whitespace=True, skiprows=4)

df.columns = [
    "id",
    "lp_disp",
    "shr_stress",
    "nor_disp",
    "nor_stress",
    "time",
    "sync",
    "samp_freq",
]

df = df.drop(["id"], axis=1)
df = df[['time'] + [col for col in df.columns if col != 'time']]

df.head()

In [ ]:
# Find the indices where target_nor_stress changes

window_size = 100
df['smoothed_nor_stress'] = df['nor_stress'].rolling(window_size, center=True).mean()
df['target_nor_stress'] = df['smoothed_nor_stress'].fillna(0).round().astype(int)
segment_indices = np.where(df['target_nor_stress'].diff() != 0)[0]

# Segment the dataframe based on the indices
segments = np.split(df, segment_indices)

In [ ]:
def detrend_first_order_poly(x, y):
    p = np.polyfit(x, y, deg=1)
    y_det = y - (p[0] * x + p[1])
    return y_det